In [1]:
from pyspark.sql.types import  *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from datetime import datetime, date
from pyspark.sql import Row
from delta import *
from delta.tables import *
import pyspark
import matplotlib.pyplot as plt

In [2]:
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '5g'), 
    ('spark.executor.cores', '2'), 
    ('spark.cores.max', '6'),
    ('spark.driver.memory','1g'),
    ('spark.executor.instances', '1'),
    ('spark.dynamicAllocation.enabled', 'true'),
    ('spark.dynamicAllocation.shuffleTracking.enabled', 'true'),
    ('spark.dynamicAllocation.executorIdleTimeout', '60s'),
    ('spark.dynamicAllocation.minExecutors', '0'),
    ('spark.dynamicAllocation.maxExecutors', '2'),
    ('spark.dynamicAllocation.initialExecutors', '1'),
    ('spark.dynamicAllocation.executorAllocationRatio', '1')])

In [3]:
builder = pyspark.sql.SparkSession.builder \
    .appName("Spark-Stream-Monitoring") \
    .master("spark://172.23.149.212:7077") \
    .config(conf=config) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/23 16:28:58 WARN Utils: Your hostname, algorand-druid-and-spark resolves to a loopback address: 127.0.0.1; using 172.23.149.212 instead (on interface ens3)
22/05/23 16:28:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c6d66a33-fda5-444f-be6e-60a4f4aa5919;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 624ms :: artifacts dl 21ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      

In [4]:
stream_df_txn = spark.readStream.format("delta").load("/mnt/delta/bronze/algod_indexer_public_txn_flat")

In [5]:
stream_df_txn.printSchema()

root
 |-- key: string (nullable = true)
 |-- ROUND: string (nullable = true)
 |-- TXID: long (nullable = true)
 |-- INTRA: long (nullable = true)
 |-- TYPEENUM: long (nullable = true)
 |-- ASSET: boolean (nullable = true)
 |-- EXTRA: string (nullable = true)
 |-- RR: long (nullable = true)
 |-- SIG: string (nullable = true)
 |-- TXN_FEE: long (nullable = true)
 |-- TXN_FV: long (nullable = true)
 |-- TXN_GH: string (nullable = true)
 |-- TXN_LV: long (nullable = true)
 |-- TXN_SND: string (nullable = true)
 |-- TXN_TYPE: string (nullable = true)
 |-- TXN_GEN: string (nullable = true)
 |-- TXN_GRP: string (nullable = true)
 |-- TXN_LX: string (nullable = true)
 |-- TXN_NOTE: string (nullable = true)
 |-- TXN_REKEY: string (nullable = true)
 |-- TXN_RCV: string (nullable = true)
 |-- TXN_AMT: string (nullable = true)
 |-- TXN_CLOSE: string (nullable = true)
 |-- TXN_VOTEKEY: string (nullable = true)
 |-- TXN_SELKEY: string (nullable = true)
 |-- TXN_VOTEFST: long (nullable = true)
 |-- T

In [6]:
query = stream_df_txn.groupBy("ROUND").count()

In [7]:
query.printSchema()

root
 |-- ROUND: string (nullable = true)
 |-- count: long (nullable = false)



In [8]:
query.writeStream.format("console").outputMode("update").start() 

22/05/23 16:29:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6840e974-53c5-4001-a77e-ec8c123ab0c1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/23 16:29:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
query_stream.stop()